In [1]:
import pandas as pd
import numpy as np
import ast

movies = pd.read_csv("movies_metadata.csv", low_memory=False)


In [3]:
movies = movies[['title', 'overview', 'genres']]
movies.dropna(inplace=True)


In [5]:
print(len(movies))


44506


In [7]:
def convert(text):
    try:
        L = []
        for i in ast.literal_eval(text):
            L.append(i['name'])
        return L
    except:
        return []

movies['genres'] = movies['genres'].apply(convert)


In [9]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())


In [11]:
movies['tags'] = movies['overview'] + movies['genres']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))


In [13]:
new_df = movies[['title','tags']]
new_df = new_df[new_df['tags'] != ""]
new_df.reset_index(drop=True, inplace=True)

print(len(new_df))


44504


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()


In [20]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend(movie):
    
    if movie not in new_df['title'].values:
        print("Movie not found!")
        return
    
    movie_index = new_df[new_df['title'] == movie].index[0]
    
    movie_vector = vectors[movie_index].reshape(1, -1)
    
    similarity_scores = cosine_similarity(movie_vector, vectors)
    
    movies_list = sorted(
        list(enumerate(similarity_scores[0])),
        reverse=True,
        key=lambda x: x[1]
    )[1:6]
    
    print("Recommended Movies:\n")
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)



In [28]:
new_df = new_df.head(10000)
new_df.reset_index(drop=True, inplace=True)


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')

vectors = cv.fit_transform(new_df['tags']).toarray()


In [32]:
print(len(new_df))
print(vectors.shape)



10000
(10000, 5000)


In [34]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend(movie):
    
    if movie not in new_df['title'].values:
        print("Movie not found!")
        return
    
    movie_index = new_df[new_df['title'] == movie].index[0]
    
    movie_vector = vectors[movie_index].reshape(1, -1)
    
    similarity_scores = cosine_similarity(movie_vector, vectors)[0]
    
    similar_movies = sorted(
        list(enumerate(similarity_scores)),
        key=lambda x: x[1],
        reverse=True
    )[1:6]
    
    print("Recommended Movies:\n")
    
    for idx, score in similar_movies:
        print(new_df.iloc[idx].title)


In [36]:
recommend("Toy Story")


Recommended Movies:

Small Fry
Hawaiian Vacation
Andy Hardy's Double Life
Gena the Crocodile
Happiness Is a Warm Blanket, Charlie Brown
